# Final Submission: Dynamic Pricing for Urban Parking Lots
Capstone Project - Summer Analytics 2025

In [1]:
!pip install pathway bokeh panel pandas numpy --quiet
import panel as pn
pn.extension('bokeh')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pathway as pw
from datetime import datetime
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource


In [16]:
# Load dataset and prepare stream CSV
df = pd.read_csv('dataset.csv')
# df.head()
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)
df = df.sort_values('Timestamp').reset_index(drop=True)
df = df.rename(columns={'SystemCodeNumber':'LocationID'})
df = df.rename(columns={'TrafficConditionNearby':'TrafficLevel'})
stream_csv = df[['Timestamp','LocationID','Capacity','Occupancy','QueueLength',
                 'TrafficLevel','IsSpecialDay','VehicleType','Latitude','Longitude']]
stream_csv.to_csv('parking_stream.csv', index=False)


In [17]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    LocationID: str
    Capacity: int
    Occupancy: int
    QueueLength: int
    TrafficLevel: str
    IsSpecialDay: int
    VehicleType: str
    Latitude: float
    Longitude: float

data = pw.demo.replay_csv('parking_stream.csv', schema=ParkingSchema, input_rate=1000)


In [19]:
import datetime as dt
# Step 1: Parse string to datetime
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime('%Y-%m-%d %H:%M:%S')
)

# Step 2: Use the parsed datetime to create a string day key
data_with_time = data_with_time.with_columns(
    day_key = data_with_time.t.dt.strftime('%Y-%m-%d') + '_' + data_with_time.LocationID
)

delta_window = (
    data_with_time.windowby(
        data_with_time.t,
        instance=data_with_time.day_key,
        window=pw.temporal.tumbling(dt.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        LocationID=pw.reducers.any(pw.this.LocationID),
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        QueueLength=pw.reducers.max(pw.this.QueueLength),
        TrafficLevel=pw.reducers.any(pw.this.TrafficLevel),
        IsSpecialDay=pw.reducers.any(pw.this.IsSpecialDay),
        VehicleType=pw.reducers.any(pw.this.VehicleType),
        Latitude=pw.reducers.max(pw.this.Latitude),
        Longitude=pw.reducers.max(pw.this.Longitude)
    )
)


In [33]:
# Pricing Models
BASE, MINP, MAXP = 10.0, 5.0, 20.0

# Baseline Model (Model 1)
occ_rate = pw.this.occ_max / pw.this.cap
price_unclipped = BASE + 0.5 * (occ_rate - 0.5)
Price_M1 = pw.if_else(
    price_unclipped < MINP, MINP,
    pw.if_else(price_unclipped > MAXP, MAXP, price_unclipped)
)
price_linear = delta_window.with_columns(Price_M1 = Price_M1)

# Demand-Based Model (Model 2)
vehicle_map = pw.if_else(
    pw.this.VehicleType == 'car', 1.0,
    pw.if_else(pw.this.VehicleType == 'bike', 0.5,
    pw.if_else(pw.this.VehicleType == 'truck', 1.5, 1.0))
)
traffic_map = pw.if_else(
    pw.this.TrafficLevel == 'low', 0.2,
    pw.if_else(pw.this.TrafficLevel == 'average', 0.5,
    pw.if_else(pw.this.TrafficLevel == 'high', 1.0, 0.5))
)

raw = (
    0.4 * occ_rate +
    0.3 * pw.this.QueueLength +
    0.2 * traffic_map +
    0.3 * (pw.this.IsSpecialDay * 1.0) +    # <-- boolean→float
    0.5 * vehicle_map
)

norm = raw / (1 + abs(raw))
price_M2_unclipped = BASE * (1 + 0.8 * norm)
Price_M2 = pw.if_else(
    norm < -1, MINP,
    pw.if_else(norm > 1, MAXP, price_M2_unclipped)
)

price_demand = delta_window.with_columns(Price_M2 = Price_M2)



In [36]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

def price_plot(source, col, title):
    """source is already a ColumnDataSource."""
    p = figure(x_axis_type='datetime', title=title, height=300)
    p.line('t', col, source=source, line_width=2)
    return p


# Use Pathway's .plot() to convert each table into DataFrame snapshots
viz = pn.Column(
    price_linear.plot(lambda dfp: price_plot(dfp, 'Price_M1', 'Model 1 – Baseline')),
    price_demand.plot(lambda dfp: price_plot(dfp, 'Price_M2', 'Model 2 – Demand-Based'))
)

viz.servable()



Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
pw.run()

Output()